In [119]:
import lightkurve as lk
import numpy as np
from apertureoptimizer import ApertureOptimizer

# Download KOI 6.01, an exoplanet False Positive
tpfs = lk.search_targetpixelfile('kepler-76', quarter=3).download_all()
tpf = tpfs[1]

# Define a corrector function
def corrector(lc):
    clc = lc.copy().flatten(501)#.remove_outliers(sigma_upper=3, sigma_lower=5)
    return clc

In [ ]:
p = tpf.to_lightcurve().flatten(501).to_periodogram('bls', minimum_period=1.4, maximum_period=1.6, frequency_factor=1, duration=0.05)

In [ ]:
mask = p.get_transit_mask()

In [ ]:
lc = tpf.to_lightcurve().flatten(501)
lc.plot()

In [ ]:
lc.fold(p.period_at_max_power.value, p.transit_time_at_max_power).scatter()

In [ ]:
corrector(tpf.to_lightcurve()).fold(period, t0).plot()

In [ ]:
# Make an ApertureOptimizer Class, pass in the False Positive parameters
a = ApertureOptimizer(tpf, period, t0, duration, corrector=corrector)

# Optimize the aperture
a.optimize()

# Plot the results
a.plot_results(bin=20)

# Use the best light curve as you normally would.
lc = a.best_lc

In [ ]:
import matplotlib.pyplot as plt
bin = 10
ax = a.starting_lc.fold(period, t0).bin(bin).errorbar(c='k', label='')
a.starting_lc.fold(period, t0).bin(bin).plot(c='k', ax=ax, label='Pipeline')


a.best_lc.fold(period, t0).bin(bin).errorbar(ax=ax, c='r', label='')
a.best_lc.fold(period, t0).bin(bin).plot(ax=ax, c='r', label='Optimal')

lk.LightCurve(a.best_lc.time, a.mask(a.best_lc.time) * 0.0005 + 1).fold(period, t0).plot(ax=ax, label='Transit Mask')
plt.xlim(-0.1, 0.1)



In [ ]:
tpf.interact(notebook_url='http://localhost:8926')